In [1]:
spark.stop()
sc.stop()

NameError: name 'spark' is not defined

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import sys
import string
from csv import reader
from functools import reduce
from pyspark.sql import functions as f
from collections import defaultdict
import datetime
%load_ext autoreload
%autoreload 2
from tableCollections import TableCollections

In [4]:
spark = SparkSession \
                .builder \
                .appName("TableCollections") \
                .config("spark.io.compression.codec", "snappy") \
                .config("spark.shuffle.service.enabled", "false") \
                .config("spark.dynamicAllocation.enabled", "false") \
                .config("spark.rdd.compress", "true") \
                .getOrCreate()
sc = spark.sparkContext

In [4]:
parkingTable = spark.read.format('csv').options(header='true',inferschema='true').load('/user/ecc290/HW1data/parking-violations-header.csv')
openTable = spark.read.format('csv').options(header='true',inferschema='true').load('/user/ecc290/HW1data/open-violations-header.csv')

In [16]:
tc = TableCollections(spark, sc)
tc.register(openTable, "open")
tc.register(parkingTable, "parking")
tc.numColWithinRange(0, 1000000000000).show()
tc.timeColWithinRange(datetime.datetime(1994,1,1), datetime.datetime(2018,5,1)).show()

num metadata file exists for table open
num metadata file exists for table parking
timestamp metadata file exists for table parking
+--------------+---------+
|       colName|tableName|
+--------------+---------+
|summons_number|     open|
|   fine_amount|     open|
|summons_number|  parking|
|violation_code|  parking|
+--------------+---------+

+----------+---------+
|   colName|tableName|
+----------+---------+
|issue_date|  parking|
+----------+---------+



In [17]:
tc = TableCollections(spark, sc)
dirname = '/user/jp4989/open_data_csv/'
found = False
with open('data_ids', 'r') as g:
    ids = g.readlines()
    for each in ids:
        if each.strip() == '54k3-2wtq':
            found = True
        if not found:
            continue
        filename = dirname+each.strip()+'.csv'
        df = spark.read.format('csv').options(header='true',inferschema='true').load(filename)
        if not df.rdd.isEmpty():
            tc.register(df, 'nyc_'+each.strip().replace('-','_'))

In [9]:
tc = TableCollections(spark, sc)
dirname = '/user/jp4989/open_data_csv/'
with open('data_ids', 'r') as g:
    ids = g.readlines()
    for each in ids:
        tc.add_registered_table_name('nyc_'+each.strip().replace('-','_'))
        if each.strip() == '6m56-5mfb':
            break

In [6]:
tc.numColWithinRange(0, 100).show()

+--------------------+-------------+
|             colName|    tableName|
+--------------------+-------------+
|City Council Dist...|nyc_22rf_yxcy|
| Community Districts|nyc_22rf_yxcy|
|  Borough Boundaries|nyc_22rf_yxcy|
|    Police Precincts|nyc_22rf_yxcy|
|         % Level 3+4|nyc_26kp_bgdh|
|           % Level 2|nyc_26kp_bgdh|
|           % Level 1|nyc_26kp_bgdh|
|           % Level 3|nyc_26kp_bgdh|
|  Average Class Size|nyc_276h_y36a|
|          Level3+4_%|nyc_27h8_t3wt|
|            Level2_%|nyc_27h8_t3wt|
|            Level3_%|nyc_27h8_t3wt|
|            district|nyc_27h8_t3wt|
|            Level4_%|nyc_27h8_t3wt|
|            Level1_%|nyc_27h8_t3wt|
|            latitude|nyc_29km_avyc|
|   Pct Level 3 and 4|nyc_2bh6_qmgg|
|         Pct Level 3|nyc_2bh6_qmgg|
|         Pct Level 4|nyc_2bh6_qmgg|
|         Pct Level 2|nyc_2bh6_qmgg|
+--------------------+-------------+
only showing top 20 rows



In [7]:
# return NYC longitude columns
tc.numColWithinRange(-76, -70).show()

+---------+-------------+
|  colName|    tableName|
+---------+-------------+
|longitude|nyc_29km_avyc|
|Longitude|nyc_2fra_mtpn|
|Longitude|nyc_2n4x_d97d|
|Longitude|nyc_2q48_ip9a|
|  Borough|nyc_35f6_8qd2|
|Longitude|nyc_35sw_rdxj|
|Longitude|nyc_36hn_wea6|
|Longitude|nyc_37fm_7uaa|
|Longitude|nyc_37it_gmcp|
|Longitude|nyc_3qfc_4tta|
|Longitude|nyc_3rfa_3xsf|
|Longitude|nyc_3spy_rjpw|
|Longitude|nyc_48pb_zy2g|
|Longitude|nyc_4wf2_7kdu|
|Longitude|nyc_4zdr_zwdi|
|Longitude|nyc_56u5_n9sa|
|Longitude|nyc_57mv_nv28|
|Longitude|nyc_59kj_x8nc|
|Longitude|nyc_59t5_r7nb|
|Longitude|nyc_5e24_x4wa|
+---------+-------------+
only showing top 20 rows



In [8]:
# return NYC latitude columns
tc.numColWithinRange(39, 43).show()

+--------+-------------+
| colName|    tableName|
+--------+-------------+
|latitude|nyc_29km_avyc|
|Latitude|nyc_2fra_mtpn|
|Latitude|nyc_2n4x_d97d|
|Latitude|nyc_2q48_ip9a|
|Latitude|nyc_35sw_rdxj|
|Latitude|nyc_36hn_wea6|
|Latitude|nyc_37fm_7uaa|
|Latitude|nyc_37it_gmcp|
|Latitude|nyc_3qfc_4tta|
|Latitude|nyc_3rfa_3xsf|
|Latitude|nyc_3spy_rjpw|
|Latitude|nyc_48pb_zy2g|
|Latitude|nyc_4wf2_7kdu|
|Latitude|nyc_4zdr_zwdi|
|Latitude|nyc_56u5_n9sa|
|Latitude|nyc_57mv_nv28|
|Latitude|nyc_59kj_x8nc|
|Latitude|nyc_59t5_r7nb|
|Latitude|nyc_5e24_x4wa|
|Latitude|nyc_5fn4_dr26|
+--------+-------------+
only showing top 20 rows



In [9]:
# return columns between 0 and 1
tc.numColWithinRange(0, 1).show()

+-------+---------+
|colName|tableName|
+-------+---------+
+-------+---------+



In [10]:
# return columns between 0 and 1
tc.numColWithinRange(-0.1, 1.1).show()

+--------------------+-------------+
|             colName|    tableName|
+--------------------+-------------+
|Unregistered Vehi...|nyc_2bnn_yakx|
|COUNT PUBLIC ASSI...|nyc_2t32_hbca|
|COUNT CITIZEN STA...|nyc_2t32_hbca|
|PERCENT GENDER UN...|nyc_2t32_hbca|
|COUNT PACIFIC ISL...|nyc_2t32_hbca|
|COUNT AMERICAN IN...|nyc_2t32_hbca|
|COUNT GENDER UNKNOWN|nyc_2t32_hbca|
|        Derived View|nyc_56b7_s9t5|
|Average Minors pe...|nyc_5r5y_pvs3|
|            Latitude|nyc_656a_faqy|
|PERCENT NRECEIVES...|nyc_6bic_qvek|
|PERCENT RECEIVES ...|nyc_6bic_qvek|
|PERCENT PUBLIC AS...|nyc_6bic_qvek|
|PERCENT PERMANENT...|nyc_6bic_qvek|
|COUNT PUBLIC ASSI...|nyc_6bic_qvek|
|PERCENT CITIZEN S...|nyc_6bic_qvek|
|PERCENT OTHER CIT...|nyc_6bic_qvek|
|COUNT CITIZEN STA...|nyc_6bic_qvek|
|PERCENT WHITE NON...|nyc_6bic_qvek|
|PERCENT ASIAN NON...|nyc_6bic_qvek|
+--------------------+-------------+
only showing top 20 rows



In [12]:
tc.getNumRange(["nyc_2n4x_d97d^Latitude", "nyc_29km_avyc^latitude", "nyc_35f6_8qd2^Borough"]).show()

+--------+----------+----------+
| colName|       min|       max|
+--------+----------+----------+
|Latitude|  40.76135| 40.769233|
|latitude| 40.508283| 40.906225|
| Borough|-74.124757|-74.022721|
+--------+----------+----------+

